In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [25]:
outages_df = pd.read_excel('data/original/Book2(1).xlsx', usecols=['Унікод АЗС', 'ДатаЧас', 'Обсяг спожитих, кВт', 'Year', 'Month', 'Date', 'Day', 'DayWeek', 'Hour'])

In [26]:
latitude = []
longitude = []

for i in outages_df['Унікод АЗС']:
    if i == 40810600:
        latitude.append(50.418366003690984)
        longitude.append(29.840546770260875)
    if i == 40861000:
        latitude.append(50.31828636812571)
        longitude.append(31.03586158987561)
    if i == 40860500:
        latitude.append(50.319652674621935)
        longitude.append(31.03398680305577)
    if i == 40810700:
        latitude.append(50.42274230627166)
        longitude.append(29.843808164165303)

outages_df['Latitude'] = latitude
outages_df['Longitude'] = longitude


In [27]:
outages_df

,Унікод АЗС,ДатаЧас,"Обсяг спожитих, кВт",Year,Month,Date,Day,DayWeek,Hour,Latitude,Longitude
0,40810600,2023-06-01 00:00:00,29.3216,2023,6,2023-06-01,1,3,0,50.418366,29.840547
1,40810600,2023-06-01 01:00:00,28.2432,2023,6,2023-06-01,1,3,1,50.418366,29.840547
2,40810600,2023-06-01 02:00:00,25.5184,2023,6,2023-06-01,1,3,2,50.418366,29.840547
3,40810600,2023-06-01 03:00:00,28.1296,2023,6,2023-06-01,1,3,3,50.418366,29.840547
4,40810600,2023-06-01 04:00:00,26.8352,2023,6,2023-06-01,1,3,4,50.418366,29.840547
...,...,...,...,...,...,...,...,...,...,...,...
49921,40810700,2024-11-01 19:00:00,59.8150,2024,11,2024-11-01,1,4,19,50.422742,29.843808
49922,40810700,2024-11-01 20:00:00,69.7740,2024,11,2024-11-01,1,4,20,50.422742,29.843808
49923,40810700,2024-11-01 21:00:00,75.8230,2024,11,2024-11-01,1,4,21,50.422742,29.843808
49924,40810700,2024-11-01 22:00:00,70.4940,2024,11,2024-11-01,1,4,22,50.422742,29.843808


In [28]:
outages_list = []
kW_list = []
for i in outages_df['Обсяг спожитих, кВт']:
    if pd.isna(i):
        outages_list.append(False)
        kW_list.append(0)
    else:
        outages_list.append(True)
        kW_list.append(i)

outages_df['Light On'] = outages_list
outages_df['kW'] = kW_list

In [29]:
outages_df = outages_df.drop(columns = ['Date'])
# outages_df = outages_df.drop(columns = ['ДатаЧас'])
outages_df = outages_df.drop(columns = ['Обсяг спожитих, кВт'])

In [44]:
X = outages_df.drop(columns=['kW'])
y = outages_df[['kW', 'Year', 'Month']] 

In [31]:
X_val = X[(X['Year'] ==2024) & (X['Month'] == 9)]
y_val = y[(y['Year'] ==2024) & (y['Month'] == 9)]

X_test = X[(X['Year'] ==2024) & (X['Month'] == 10)]
y_test = y[(y['Year'] ==2024) & (y['Month'] == 10)]

X_train = X[~((X['Year'] == 2024) & (X['Month'] == 10))]
y_train = y[~((y['Year'] == 2024) & (y['Month'] == 10))]

X_train = X_train[(~((X_train['Year'] == 2024) & (X_train['Month'] == 9))) | (~((y_train['Year'] == 2024) & (y_train['Month'] == 10)))]

In [32]:
X_val = X_val.drop(columns = ['ДатаЧас'])
y_val = y_val.drop(columns = ['Year', 'Month'])

X_test = X_test.drop(columns = ['ДатаЧас'])
y_test = y_test.drop(columns = ['Year', 'Month'])

X_train = X_train.drop(columns = ['ДатаЧас'])
y_train = y_train.drop(columns = ['Year', 'Month'])

In [33]:
y_test

,kW
11713,36.7440
11714,36.9056
11715,38.0320
11716,39.5776
11717,37.9488
...,...
49897,97.1970
49898,98.7220
49899,70.2430
49900,50.0960


In [34]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm

In [41]:
err

26.442543799714702

In [14]:
X_val

,Унікод АЗС,Year,Month,Day,DayWeek,Hour,Latitude,Longitude,Light On
11713,40810600,2024,10,1,1,0,50.418366,29.840547,True
11714,40810600,2024,10,1,1,1,50.418366,29.840547,True
11715,40810600,2024,10,1,1,2,50.418366,29.840547,True
11716,40810600,2024,10,1,1,3,50.418366,29.840547,True
11717,40810600,2024,10,1,1,4,50.418366,29.840547,True
...,...,...,...,...,...,...,...,...,...
49897,40810700,2024,10,31,3,19,50.422742,29.843808,True
49898,40810700,2024,10,31,3,20,50.422742,29.843808,True
49899,40810700,2024,10,31,3,21,50.422742,29.843808,True
49900,40810700,2024,10,31,3,22,50.422742,29.843808,True


In [42]:
import warnings
from tqdm import tqdm
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd
import logging

# Suppress warnings and LightGBM logs
warnings.filterwarnings("ignore")
logging.getLogger("lightgbm").setLevel(logging.CRITICAL)

# Hyperparameter grids
n_estimators_list = [50, 100, 200, 500, 1000, 2000]
max_depth_list = [4, 6, 8, 10, 12, 16]
learning_rate_list = [0.01, 0.05, 0.1, 0.2]
num_leaves_list = [10, 20, 30, 40, 50]

# Flatten all combinations of parameters into one list
param_combinations = [(n_estimators, max_depth, learning_rate, num_leaves)
                      for n_estimators in n_estimators_list
                      for max_depth in max_depth_list
                      for learning_rate in learning_rate_list
                      for num_leaves in num_leaves_list]

# Best model parameters
best_n_estimators = None
best_max_depth = None
best_learning_rate = None
best_num_leaves = None
best_rmse = float('inf')
best_lgbm_model = None

# Single progress bar for all iterations
for n_estimators, max_depth, learning_rate, num_leaves in tqdm(param_combinations, desc="Grid search progress"):
    lgbm_model = LGBMRegressor(n_estimators=n_estimators,
                               max_depth=max_depth,
                               learning_rate=learning_rate,
                               num_leaves=num_leaves,
                               verbose=-1,  # Suppress training logs
                               n_jobs=-1,
                               random_state=42)
    lgbm_model.fit(X_train, y_train)
    validation_preds = lgbm_model.predict(X_val)
    validation_preds = pd.Series(validation_preds)
    validation_target = pd.Series(y_val['kW'].values)
    err = np.sqrt(mse(validation_target, validation_preds))
    
    # Update the best parameters if RMSE improves
    if err < best_rmse:
        best_rmse = err
        best_n_estimators = n_estimators
        best_max_depth = max_depth
        best_learning_rate = learning_rate
        best_num_leaves = num_leaves
        best_lgbm_model = lgbm_model

# Print the best results
print(f'LightGBM RMSE: {best_rmse}')
print(f'n_estimators: {best_n_estimators}, max_depth: {best_max_depth}, '
      f'learning_rate: {best_learning_rate}, num_leaves: {best_num_leaves}')


Grid search progress: 100%|██████████| 720/720 [28:03<00:00,  2.34s/it]

LightGBM RMSE: 14.114658749586626
n_estimators: 2000, max_depth: 12, learning_rate: 0.2, num_leaves: 50


In [43]:
test_preds = lgbm_model.predict(X_test)
test_preds = pd.Series(test_preds)
test_target = pd.Series(y_test['kW'].values)
err = np.sqrt(mse(test_target, test_preds))
err

26.577200697029728

In [13]:
import warnings
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd
import logging

# Suppress warnings and XGBoost logs
warnings.filterwarnings("ignore")
logging.getLogger("xgboost").setLevel(logging.CRITICAL)

# Hyperparameter grids
n_estimators_list = [50, 100, 200, 500, 1000]
max_depth_list = [4, 6, 8, 10, 12, 16, 18, 20]
learning_rate_list = [0.01, 0.05, 0.1, 0.2]
subsample_list = [0.8, 1.0]

# Flatten all combinations of parameters into one list
param_combinations = [(n_estimators, max_depth, learning_rate, subsample)
                      for n_estimators in n_estimators_list
                      for max_depth in max_depth_list
                      for learning_rate in learning_rate_list
                      for subsample in subsample_list]

# Best model parameters
best_n_estimators = None
best_max_depth = None
best_learning_rate = None
best_subsample = None
best_rmse = float('inf')
best_xgb_model = None
best_preds = None

# Single progress bar for all iterations
for n_estimators, max_depth, learning_rate, subsample in tqdm(param_combinations, desc="Grid search progress"):
    xgb_model = xgb.XGBRegressor(n_estimators=n_estimators,
                                 max_depth=max_depth,
                                 learning_rate=learning_rate,
                                 subsample=subsample,
                                 n_jobs=-1,
                                 verbosity=0,  # Suppress training logs
                                 random_state=42)
    xgb_model.fit(X_train, y_train)
    validation_preds = xgb_model.predict(X_val)
    validation_preds = pd.Series(validation_preds)
    validation_target = pd.Series(y_val['kW'].values)
    err = np.sqrt(mse(validation_target, validation_preds))
    
    # Update the best parameters if RMSE improves
    if err < best_rmse:
        best_rmse = err
        best_n_estimators = n_estimators
        best_max_depth = max_depth
        best_learning_rate = learning_rate
        best_subsample = subsample
        best_xgb_model = xgb_model
        best_preds = validation_preds

# Print the best results
print(f'XGBoost RMSE: {best_rmse}')
print(f'n_estimators: {best_n_estimators}, max_depth: {best_max_depth}, '
      f'learning_rate: {best_learning_rate}, subsample: {best_subsample}')


Grid search progress: 100%|██████████| 320/320 [46:24<00:00,  8.70s/it]  

XGBoost RMSE: 0.2411003406127406
n_estimators: 1000, max_depth: 20, learning_rate: 0.1, subsample: 1.0


In [22]:
xgb_model = xgb.XGBRegressor(n_estimators=2000,
                                 max_depth=50,
                                 learning_rate=0.1,
                                 subsample=1,
                                 n_jobs=-1,
                                 verbosity=0)
xgb_model.fit(X_train, y_train)
validation_preds = xgb_model.predict(X_val)
validation_preds = pd.Series(validation_preds)
validation_target = pd.Series(y_val['kW'].values)
np.sqrt(mse(validation_target, validation_preds))

0.24109530141678084

In [45]:
X['Night'] = X['Hour'].apply(lambda x: True if (x >= 23 or x <= 6) else False)

In [46]:
X_val = X[(X['Year'] ==2024) & (X['Month'] == 9)]
y_val = y[(y['Year'] ==2024) & (y['Month'] == 9)]

X_test = X[(X['Year'] ==2024) & (X['Month'] == 10)]
y_test = y[(y['Year'] ==2024) & (y['Month'] == 10)]

X_train = X[~((X['Year'] == 2024) & (X['Month'] == 10))]
y_train = y[~((y['Year'] == 2024) & (y['Month'] == 10))]

X_train = X_train[(~((X_train['Year'] == 2024) & (X_train['Month'] == 9))) | (~((y_train['Year'] == 2024) & (y_train['Month'] == 10)))]

In [47]:
X_val = X_val.drop(columns = ['ДатаЧас'])
y_val = y_val.drop(columns = ['Year', 'Month'])

X_test = X_test.drop(columns = ['ДатаЧас'])
y_test = y_test.drop(columns = ['Year', 'Month'])

X_train = X_train.drop(columns = ['ДатаЧас'])
y_train = y_train.drop(columns = ['Year', 'Month'])